In [41]:
import pandas as pd
import geopandas as gpd
import os
import requests

In [62]:
# Step 1: Read the semicolon-delimited CSV
df = pd.read_csv("C:/Users/edoar/Downloads/ds1996_quotazioni_omi_compravendita_e_locazione_riepilogo.csv", sep=";")

# Step 2: Filter rows where `Anno == 2024`
df_2024 = df[df["Anno"] == 2024].copy()  # Create a copy to avoid the warning

# Step 3: Filter rows based on `Descr_Tipologia`
valid_types = ["Abitazioni civili", "Abitazioni di tipo economico", "Abitazioni signorili", "Ville e Villini"]
filtered_df = df_2024[df_2024["Descr_Tipologia"].isin(valid_types)].copy()  # Create another copy

# Step 4: Calculate the mean between `Compr_min` and `Compr_max`
filtered_df["Compr_mean"] = (filtered_df["Compr_min"] + filtered_df["Compr_max"]) / 2

filtered_df

,Anno,Periodo,Area_territoriale,Regione,Prov,Comune_ISTAT,Comune_cat,Sez,Comune_amm,Comune_descrizione,...,Descr_Tipologia,Stato,Stato_prev,Compr_min,Compr_max,Sup_NL_compr,Loc_min,Loc_max,Sup_NL_loc,Compr_mean
25460,2024,1 semestre,NORD-OVEST,LOMBARDIA,MI,3015146,C1AA,,F205,MILANO,...,Abitazioni civili,OTTIMO,,10400,14000,L,"32,1",41,L,12200.0
25461,2024,1 semestre,NORD-OVEST,LOMBARDIA,MI,3015146,C1AA,,F205,MILANO,...,Abitazioni civili,NORMALE,P,8000,10300,L,25,32,L,9150.0
25462,2024,1 semestre,NORD-OVEST,LOMBARDIA,MI,3015146,C1AA,,F205,MILANO,...,Abitazioni di tipo economico,NORMALE,P,6800,8400,L,20,26,L,7600.0
25463,2024,1 semestre,NORD-OVEST,LOMBARDIA,MI,3015146,C1AA,,F205,MILANO,...,Abitazioni di tipo economico,OTTIMO,,8500,9700,L,"26,1",31,L,9100.0
25464,2024,1 semestre,NORD-OVEST,LOMBARDIA,MI,3015146,C1AA,,F205,MILANO,...,Abitazioni signorili,OTTIMO,P,12500,17000,L,35,47,L,14750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25925,2024,1 semestre,NORD-OVEST,LOMBARDIA,MI,3015146,C1AA,,F205,MILANO,...,Abitazioni di tipo economico,NORMALE,P,1500,1950,L,4,"5,5",L,1725.0
25934,2024,1 semestre,NORD-OVEST,LOMBARDIA,MI,3015146,C1AA,,F205,MILANO,...,Abitazioni civili,OTTIMO,,2450,3300,L,"8,5",13,L,2875.0
25935,2024,1 semestre,NORD-OVEST,LOMBARDIA,MI,3015146,C1AA,,F205,MILANO,...,Abitazioni civili,NORMALE,P,1700,2400,L,"6,5","8,4",L,2050.0
25936,2024,1 semestre,NORD-OVEST,LOMBARDIA,MI,3015146,C1AA,,F205,MILANO,...,Abitazioni di tipo economico,OTTIMO,,2100,2500,L,"6,6","8,5",L,2300.0


In [64]:
# URL of the GeoJSON file
geojson_url = "https://dati.comune.milano.it/dataset/37e0fb6c-521e-4ecf-9043-f3bfa9400d6d/resource/cba1ced9-532a-480e-845d-8875ff9cb538/download/_expl_f205-2024_1.geojson"

# Directory to save the GeoJSON file
output_directory = "geojson_files"

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# File name and output path
file_name = geojson_url.split("/")[-1]
output_path = os.path.join(output_directory, file_name)

try:
    print(f"Downloading {geojson_url}...")
    response = requests.get(geojson_url)
    response.raise_for_status()  # Raise an error for HTTP issues

    # Save the GeoJSON file
    with open(output_path, "wb") as file:
        file.write(response.content)

    print(f"Saved {file_name} to {output_directory}")

except requests.exceptions.RequestException as e:
    print(f"Failed to download {geojson_url}: {e}")


Saved _expl_f205-2024_1.geojson to geojson_files


In [65]:
# Step 1: Load the GeoJSON dataset
PolyZones = gpd.read_file("geojson_files/_expl_f205-2024_1.geojson")

# Merge the filtered CSV with the GeoJSON on 'Zona'
filtered_df['Zona'] = filtered_df['Zona'].astype(str)
PolyZones['Zona'] = PolyZones['Zona'].astype(str)
merged_gdf = PolyZones.merge(filtered_df, on="Zona", how="left")
merged_gdf

,Name,Anno / Semestre,Fascia_x,Zona_Descr,Zona,LinkZona_x,Cod_tip_prev,Descr_tip_prev,Stato_prev_x,Microzona,...,Descr_Tipologia,Stato,Stato_prev_y,Compr_min,Compr_max,Sup_NL_compr,Loc_min,Loc_max,Sup_NL_loc,Compr_mean
0,MILANO - Zona OMI B12,2024 / 1,B,"CENTRO STORICO -DUOMO, SANBABILA, MONTENAPOLEO...",B12,MI00003228,20,Abitazioni civili,N,2,...,Abitazioni civili,OTTIMO,,10400.0,14000.0,L,"32,1",41,L,12200.0
1,MILANO - Zona OMI B12,2024 / 1,B,"CENTRO STORICO -DUOMO, SANBABILA, MONTENAPOLEO...",B12,MI00003228,20,Abitazioni civili,N,2,...,Abitazioni civili,NORMALE,P,8000.0,10300.0,L,25,32,L,9150.0
2,MILANO - Zona OMI B12,2024 / 1,B,"CENTRO STORICO -DUOMO, SANBABILA, MONTENAPOLEO...",B12,MI00003228,20,Abitazioni civili,N,2,...,Abitazioni di tipo economico,NORMALE,P,6800.0,8400.0,L,20,26,L,7600.0
3,MILANO - Zona OMI B12,2024 / 1,B,"CENTRO STORICO -DUOMO, SANBABILA, MONTENAPOLEO...",B12,MI00003228,20,Abitazioni civili,N,2,...,Abitazioni di tipo economico,OTTIMO,,8500.0,9700.0,L,"26,1",31,L,9100.0
4,MILANO - Zona OMI B12,2024 / 1,B,"CENTRO STORICO -DUOMO, SANBABILA, MONTENAPOLEO...",B12,MI00003228,20,Abitazioni civili,N,2,...,Abitazioni signorili,OTTIMO,P,12500.0,17000.0,L,35,47,L,14750.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,MILANO - Zona OMI E8,2024 / 1,E,"QUARTO OGGIARO, SACCO",E8,MI00004787,20,Abitazioni civili,N,0,...,Abitazioni civili,OTTIMO,,2450.0,3300.0,L,"8,5",13,L,2875.0
172,MILANO - Zona OMI E8,2024 / 1,E,"QUARTO OGGIARO, SACCO",E8,MI00004787,20,Abitazioni civili,N,0,...,Abitazioni civili,NORMALE,P,1700.0,2400.0,L,"6,5","8,4",L,2050.0
173,MILANO - Zona OMI E8,2024 / 1,E,"QUARTO OGGIARO, SACCO",E8,MI00004787,20,Abitazioni civili,N,0,...,Abitazioni di tipo economico,OTTIMO,,2100.0,2500.0,L,"6,6","8,5",L,2300.0
174,MILANO - Zona OMI E8,2024 / 1,E,"QUARTO OGGIARO, SACCO",E8,MI00004787,20,Abitazioni civili,N,0,...,Abitazioni di tipo economico,NORMALE,P,1500.0,2000.0,L,"5,2","6,5",L,1750.0


In [68]:
# Step 1: Import of the Polygons
gdf_combined = gpd.read_file("C:/Users/edoar/combined_quartieri.geojson")

merged_gdf = merged_gdf.to_crs(gdf_combined.crs)

# Perform the spatial join between merged_gdf (Zona polygons) and gdf_combined (Neighborhoods)
joined_gdf = gpd.sjoin(gdf_combined, merged_gdf, how="inner", predicate="intersects")
joined_gdf

,Neighborhood,geometry,index_right,Name,Anno / Semestre,Fascia_x,Zona_Descr,Zona,LinkZona_x,Cod_tip_prev,...,Descr_Tipologia,Stato,Stato_prev_y,Compr_min,Compr_max,Sup_NL_compr,Loc_min,Loc_max,Sup_NL_loc,Compr_mean
0,Parco delle Abbazie,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ...",175,MILANO - Zona OMI R2,2024 / 1,R,"RONCHETTO, CHIARAVALLE, RIPAMONTI",R2,MI00004391,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Parco delle Abbazie,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ...",101,MILANO - Zona OMI D18,2024 / 1,D,"MAROCCHETTI, VIGENTINO, CHIESA ROSSA",D18,MI00000325,20,...,Abitazioni civili,OTTIMO,,3600.0,5100.0,L,"13,1",18,L,4350.0
0,Parco delle Abbazie,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ...",104,MILANO - Zona OMI D18,2024 / 1,D,"MAROCCHETTI, VIGENTINO, CHIESA ROSSA",D18,MI00000325,20,...,Abitazioni di tipo economico,NORMALE,P,2150.0,2500.0,L,"6,8","7,9",L,2325.0
0,Parco delle Abbazie,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ...",102,MILANO - Zona OMI D18,2024 / 1,D,"MAROCCHETTI, VIGENTINO, CHIESA ROSSA",D18,MI00000325,20,...,Abitazioni civili,NORMALE,P,2350.0,3500.0,L,"7,3",13,L,2925.0
0,Parco delle Abbazie,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ...",103,MILANO - Zona OMI D18,2024 / 1,D,"MAROCCHETTI, VIGENTINO, CHIESA ROSSA",D18,MI00000325,20,...,Abitazioni di tipo economico,OTTIMO,,2600.0,3100.0,L,8,10,L,2850.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,Forze Armate,"POLYGON ((9.09753 45.46286, 9.09607 45.4625, 9...",157,MILANO - Zona OMI E5,2024 / 1,E,"BAGGIO, Q. ROMANO, MUGGIANO",E5,MI00004784,20,...,Abitazioni civili,OTTIMO,,2900.0,3500.0,L,"10,6",14,L,3200.0
84,Forze Armate,"POLYGON ((9.09753 45.46286, 9.09607 45.4625, 9...",115,MILANO - Zona OMI D24,2024 / 1,D,"SEGESTA, ARETUSA, VESPRI SICILIANI",D24,MI00000331,20,...,Abitazioni di tipo economico,OTTIMO,P,2700.0,3200.0,L,"9,6",12,L,2950.0
84,Forze Armate,"POLYGON ((9.09753 45.46286, 9.09607 45.4625, 9...",113,MILANO - Zona OMI D24,2024 / 1,D,"SEGESTA, ARETUSA, VESPRI SICILIANI",D24,MI00000331,20,...,Abitazioni civili,OTTIMO,,4300.0,6300.0,L,"14,6",21,L,5300.0
84,Forze Armate,"POLYGON ((9.09753 45.46286, 9.09607 45.4625, 9...",114,MILANO - Zona OMI D24,2024 / 1,D,"SEGESTA, ARETUSA, VESPRI SICILIANI",D24,MI00000331,20,...,Abitazioni civili,NORMALE,P,2600.0,4300.0,L,9,"14,5",L,3450.0


In [82]:
# Step 4: Group by Neighborhood and calculate desired statistics
final_gdf = joined_gdf.groupby("Neighborhood").agg({
    "Compr_min": "min",            # Minimum Compr_min for each Neighborhood
    "Compr_max": "max",            # Maximum Compr_max for each Neighborhood
    "Compr_mean": "mean",          # Average Compr_mean for each Neighborhood
    "geometry": "first"            # Representative geometry of the neighborhood
}).reset_index()

# Convert back to GeoDataFrame
final_gdf = gpd.GeoDataFrame(final_gdf, geometry="geometry", crs=gdf_combined.crs)

# Saving the file
final_gdf.to_file("PolyHomePrices.geojson", driver="GeoJSON")
final_gdf

,Neighborhood,Compr_min,Compr_max,Compr_mean,geometry
0,Adriano,2000.0,4900.0,3178.125000,"POLYGON ((9.25524 45.51983, 9.2553 45.52041, 9..."
1,Affori,1700.0,6200.0,3167.857143,"POLYGON ((9.18204 45.50732, 9.18161 45.50773, ..."
2,Baggio,1650.0,4400.0,2775.000000,"POLYGON ((9.08347 45.45145, 9.0836 45.45152, 9..."
3,Bande Nere,2000.0,11000.0,4710.714286,"POLYGON ((9.12906 45.46064, 9.12965 45.45953, ..."
4,Barona,2000.0,5000.0,3193.750000,"POLYGON ((9.16227 45.42787, 9.16288 45.42869, ..."
...,...,...,...,...,...
80,Viale Monza,2100.0,4900.0,3312.500000,"POLYGON ((9.22272 45.52373, 9.2222 45.5232, 9...."
81,Vigentina,2500.0,12500.0,5980.681818,"POLYGON ((9.19652 45.45204, 9.19803 45.45204, ..."
82,Villapizzone,1500.0,11000.0,3912.500000,"POLYGON ((9.14737 45.50568, 9.14746 45.5065, 9..."
83,Washington,2100.0,11000.0,5359.722222,"POLYGON ((9.15716 45.45665, 9.15801 45.45691, ..."
